In [17]:
from pathlib import Path
import sys
import numpy as np
from scipy.constants import g

# Ensure personal Python packages are in path
if (ppfolder := str(Path('..') / 'personal_python_packages')) not in sys.path:
    sys.path.append(ppfolder)
from LNECSPA import LTFdb

def ltf_to_txt(file_path):
    """
    Read LNEC .acq file and export to TXT, automatically classifying
    channels as displacement or acceleration based on types or units.
    Displacement units: 'mm' -> convert to meters.
    Acceleration units: 'g'  -> convert to m/s^2.
    After conversion, update units to SI and fill missing types.
    """
    file_path = Path(file_path)
    tgtA = LTFdb()
    tgtA.read(file_path)

    # Determine channel indices by type or unit
    acc_inds = []
    disp_inds = []
    for idx, (typ, unit) in enumerate(zip(tgtA.types, tgtA.units)):
        t = (typ or '').strip().lower()
        u = (unit or '').strip().lower()
        if t == 'acceleration' or (not t and u == 'g'):
            acc_inds.append(idx)
            # Fill missing type
            if not tgtA.types[idx].strip():
                tgtA.types[idx] = 'acceleration'
        elif t == 'displacement' or (not t and u == 'mm'):
            disp_inds.append(idx)
            # Fill missing type
            if not tgtA.types[idx].strip():
                tgtA.types[idx] = 'displacement'
        else:
            # Channels with unknown type/unit will be skipped or handled here
            print(f"Channel {idx}: Unknown type '{typ}' and/or unit '{unit}'. Skipping.")
            continue

    # Extract data arrays
    disp_data = tgtA.data[disp_inds].astype(float) if disp_inds else None
    acc_data = tgtA.data[acc_inds].astype(float) if acc_inds else None

    # Convert to SI units and update units
    if disp_data is not None:
        for out_idx, ch_idx in enumerate(disp_inds):
            original_unit = tgtA.units[ch_idx].strip().lower()
            if original_unit == 'mm':
                disp_data[out_idx] *= 1e-3  # mm to m
                tgtA.units[ch_idx] = 'm'
            elif original_unit == 'm':
                # already SI
                pass
            else:
                print(f"Displacement channel {ch_idx}: Unrecognized unit '{original_unit}'. No conversion applied.")

    if acc_data is not None:
        for out_idx, ch_idx in enumerate(acc_inds):
            original_unit = tgtA.units[ch_idx].strip().lower()
            if original_unit == 'g':
                acc_data[out_idx] *= g  # g to m/s^2
                tgtA.units[ch_idx] = 'm/s^2'
            elif original_unit in ['m/s^2', 'm/s²']:
                # already SI
                pass
            else:
                print(f"Acceleration channel {ch_idx}: Unrecognized unit '{original_unit}'. No conversion applied.")

    # Assign back converted data
    if disp_data is not None:
        tgtA.data[disp_inds] = disp_data
    if acc_data is not None:
        tgtA.data[acc_inds] = acc_data

    # Write output text file
    filename = file_path.name
    tgtA.write_txt(filename+str(".txt"))


In [ ]:
file_path = r'C:\Users\afons\OneDrive - Universidade de Lisboa\Controlo de Plataforma Sismica\LNEC_Adapta_Driver\LNEC_ERIES_RE-SAFE\CTL\TestLAquila\LAquilaReducedScale.tgt'
ltf_to_txt(file_path)

file_path = r'C:\Users\afons\OneDrive - Universidade de Lisboa\Controlo de Plataforma Sismica\LNEC_Adapta_Driver\LNEC_ERIES_RE-SAFE\CTL\TestLAquila\2024-09-11\LAquilaReducedScale_34.DRV'
ltf_to_txt(file_path)

file_path = r'C:\Users\afons\OneDrive - Universidade de Lisboa\Controlo de Plataforma Sismica\LNEC_Adapta_Driver\LNEC_ERIES_RE-SAFE\CTL\TestLAquila\2024-09-11\LAquilaReducedScale_34.acq'
ltf_to_txt(file_path)